# Discrete Fourier Transform example

Display the DFT and FFT (for comparison) of a 1 Hz cosine.

PARAMETERS :

    - Periods : Number of cosine periods
    - Sampling : Sampling frequency
    - 0-padding : Number of zero samples to be added at the end of signal (for better spectral interpolation)

In [14]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import time

from ipywidgets import interactive, interact, widgets, HBox, VBox, Layout


# Display the DFT and FFT (for comparison) of a 1 Hz cosine.
# PARAMETERS OF UI:
    # Periods : Number of cosine periods
    # Sampling (Hz) : Sampling frequency
    # 0-padding : Number of zero samples to be added at the end of signal (for better spectral interpolation)

    
def fourierTransform(numberOfPeriods, samplesPerPeriod, zeroPadding):
    
    # VARIABLE INITIALIZATION #
    
    # Signal variables
    T = 1.0 # period of signal (s)  
    Np = numberOfPeriods # number of desired periods
    omega = 2*(np.pi)/T
    
    # Sampling variables
    fs = samplesPerPeriod/T # sampling frequency (Hz)
    N = int(Np*T*fs) + zeroPadding # number of samples
    
    # Creation of time and frequency axis
    t = (1/fs)*np.arange(0, N, 1)
    f = (fs/N)*np.arange(-(N/2)+1,N/2+1,1)
    
    
    # SIGNAL DEFINITION #
    
    signal = np.concatenate([(np.cos(omega*t))[0:N-zeroPadding],np.zeros(zeroPadding)]) # Signal and its zero padding
    
    
    # DFT COMPUTATION #

    # Vandermonde-Fourier Matrix for DFT computation
    baseFourier = np.exp(-2j*(np.pi)/N)
    arrayFourier = np.power(np.full(N, baseFourier), np.arange(0, N, 1))
    matrixFourier = np.vander(arrayFourier, N, 1)
    
    # DFT computation
    DFT = matrixFourier.dot(signal)
    normalizedMagnitudeDFT = (np.absolute(DFT))/(N-zeroPadding)
    
    
    # NUMPY FFT COMPUTATION #
    
    normalizedMagnitudeFFT = np.absolute(np.fft.fft(signal))/(N-zeroPadding)
    
    
    # DISPLAYING RESULTS #
    
    # Signal
    ax1.clear()
    ax1.plot(t, signal)
    ax1.set_title('Signal')
    ax1.set_xlabel("t (s)")
    
    # DFT
    ax2.clear()
    ax2.plot(f, np.concatenate([(normalizedMagnitudeDFT)[int((N/2))+1:N],(normalizedMagnitudeDFT)[0:int((N/2))+1]]))
    ax2.set_title('DFT')
    ax2.set_xlabel('f (Hz)')
    
    # FFT 
    ax3.clear()
    ax3.plot(f,np.concatenate([(normalizedMagnitudeFFT)[int((N/2))+1:N],(normalizedMagnitudeFFT)[0:int((N/2))+1]]))
    ax3.set_title('FFT')
    ax3.set_xlabel('f (Hz)')


# CREATE INTERACTIBLE GRAPH #

style = {'description_width': 'initial'} # To prevent slider text collapsing

# Creation of sliders
numberOfPeriodsWidget = widgets.IntSlider(min=1, max=50, step=1, value=2, description="Periods", style=style, continuous_update=False) 
samplesPerPeriodWidget = widgets.IntSlider(min=2, max=20, step=2, value=4, description="Sampling (Hz)", style=style, continuous_update=False) 
zeroPaddingWidget = widgets.IntSlider(min=0, max=500, step=1, value=0, description="0-Padding", style=style, continuous_update=False)

# No matter chosen period and sampling, zero padding can represent up to 90% of signal length
def update_zeroPadding(*args):
    zeroPaddingWidget.max = 9*(numberOfPeriodsWidget.value)*(samplesPerPeriodWidget.value)
    zeroPaddingWidget.step = (zeroPaddingWidget.max)/30

numberOfPeriodsWidget.observe(update_zeroPadding, 'value')
samplesPerPeriodWidget.observe(update_zeroPadding, 'value')

# Creating figure inside an "Output widget"
output = widgets.Output()

with output :
    fig, (ax1, ax2, ax3) = plt.subplots(constrained_layout=True, nrows=3, ncols=1)
    fig.suptitle('DFT and FFT of 1 Hz cosine', fontsize=16)
    fig.canvas.header_visible = False

# Linking sliders with function it controls ('fourierTransform')
widget = interactive(fourierTransform, numberOfPeriods = numberOfPeriodsWidget,
         samplesPerPeriod = samplesPerPeriodWidget, 
         zeroPadding = zeroPaddingWidget)

# We run 'fourierTransform' before displaying widget so graphs are not empty 
fourierTransform(numberOfPeriodsWidget.value, samplesPerPeriodWidget.value, zeroPaddingWidget.value)

# Display sliders and graphs with proper layout
box_layout = Layout(display='flex',
                    align_items='center')
controls = VBox(widget.children[:-1])
display(HBox([output, controls], layout=box_layout))